In [34]:
HTML(read(open("style.css"), String))

LoadError: SystemError: opening file "style.css": No such file or directory

# Play a Chess Game

In [35]:
## TODO DONT Open this file twice

In [36]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("NBInclude")
using NBInclude

`printGame(game)` prints the current position of a game.

In [37]:
function printGame(game)
    display(board(game))
end

printGame (generic function with 1 method)

Import Notebooks needed.

In [38]:
@nbinclude("Random Chess.ipynb")

LoadError: SystemError: opening file "C:\\Users\\yiwen\\Documents\\GitHub\\ChessAIJulia\\Games\\Random Chess.ipynb": No such file or directory

In [39]:
@nbinclude("PGN_Export.ipynb")

LoadError: SystemError: opening file "C:\\Users\\yiwen\\Documents\\GitHub\\ChessAIJulia\\Games\\PGN_Export.ipynb": No such file or directory

In [40]:
@nbinclude("ValidateUserInput.ipynb")

LoadError: SystemError: opening file "C:\\Users\\yiwen\\Documents\\GitHub\\ChessAIJulia\\Games\\ValidateUserInput.ipynb": No such file or directory

In [41]:
@nbinclude("EvaluatePosition.ipynb")

LoadError: SystemError: opening file "C:\\Users\\yiwen\\Documents\\GitHub\\ChessAIJulia\\Games\\EvaluatePosition.ipynb": No such file or directory

In [42]:
@nbinclude("Minimax.ipynb")

LoadError: SystemError: opening file "C:\\Users\\yiwen\\Documents\\GitHub\\ChessAIJulia\\Games\\Minimax.ipynb": No such file or directory

In [43]:
@nbinclude("AlphaBetaPruning.ipynb")

LoadError: SystemError: opening file "C:\\Users\\yiwen\\Documents\\GitHub\\ChessAIJulia\\Games\\AlphaBetaPruning.ipynb": No such file or directory

In [44]:
@nbinclude("IterativeDeepening.ipynb")

LoadError: SystemError: opening file "C:\\Users\\yiwen\\Documents\\GitHub\\ChessAIJulia\\Games\\IterativeDeepening.ipynb": No such file or directory

Init Cache

In [45]:
gCache = Dict()

Dict{Any, Any}()

The function `setWin(game)` takes in a `game` in a terminal position and sets the `result` value of the game and prints the result of the game as a text message.

In [46]:
function setWin(game)
    if ischeckmate(board(game)) # decicive game
        color = sidetomove(board(game))
        color == WHITE ? color = "BLACK" : color = "WHITE"
        setGameResult(game, color)
        print(color * " wins by checkmate")
    else # draw
        if isstalemate(board(game))
            print("Stalemate")
        elseif ismaterialdraw(board(game))
            print("Draw by insufficient material")
        elseif isrule50draw(board(game))
            print("Draw by 50 move draw")
        end
        setGameResult(game, "draw")
    end
    
end

setWin (generic function with 1 method)

The function `clearCache(board, move)` takes in the current `board` and the `move` that will be made next and clears the global Cache `gCache` if the move done is a pawn move or a capture. A chess position will not occure again if one of those two events happen and therefore all entries in the Cache will not be needed anymore. This will improve the runtime slightly.

In [47]:
function clearCache(board, move)
    global gCache
    if(ptype(pieceon(board, from(move))) == PAWN || ptype(pieceon(board, to(move))) != EMPTY)
        println("Cleared Cache with $(length(gCache))")
        gCache = Dict()
    end
end

clearCache (generic function with 1 method)

The function `playMove` is a helping function for the main `playGame` function and takes in the played game and it's current static evaluation. It applies an engine move to the game and information about the current evaluation of the engine, the static evaluation and the length of the cache. The function returns the new static evaluation as `current_boardscore` of the board after applying the engine move.

Possible engines are the `Minimax Algorithm`, `Alpha-Beta-Pruning Algorithm` and `Iterative Deepening Algorithm`.

In [48]:
function playMove(game, current_boardscore)
    # MINIMAX ENGINE
    # eval, move = minimax(board(game), currentcurrent_boardscorescore, 4)

    # ALPHA BETA PRUNING ENGINE
    # eval, move = alphaBetaPruning(board(game), current_boardscore, 4)

    # ITERATIVE DEEPENING ENGINE
    eval, move = iterativeDeepening(board(game), current_boardscore, 4)

    println("Evaluation of engine: $eval")
    println("Current board score: $current_boardscore")
    println("Current entries in Cache: $(length(gCache))")
    clearCache(board(game), move)
    current_boardscore = evaluate_move(board(game), move, current_boardscore)
    domove!(game, move)
    return current_boardscore
end

playMove (generic function with 1 method)

## Main function

The function `playGame()` allows a user to play a chess game. The AI is set via the `playMove(game)` function. After the game is finished (checkmate or draw) the game is saved as a PGN-file and saved in the /Games directory. The `debug` parameter can be set to `true` to print debugging statements, default is `false`.

The function asks the user to input their name and what color they want to play as at the beginning. After that the player and the engine take alternate moves playing a chess game. 

Moves are inputted as strings which contain the pieces current location and the pieces end location. For example playing `1. e4` as white requires the string `e2e4` as input.

To resign the user can type `resign` as his move.

In [49]:
function playGame(debug = false)
    # Setup Board
    game = Game()
    printGame(game)
    setGameHeaders(game)
    current_boardscore = 0  # evaluate_position(startboard())
    
    println("What's your name?")
    username = readline()
    
    println("Do you want to play as 'white' or 'black'?")
    color = readline()
    
    # Offset move if player is black
    if color != "black" && color != "white"
        print("Invalid color. Choose white or black")   
        return
    elseif color == "black"
        setheadervalue!(game, "Black", username)
        current_boardscore = playMove(game, current_boardscore)
        printGame(game)
    elseif color == "white"
        setheadervalue!(game, "White", username)
    end

    while true
        # User Move
        if !isterminal(game)
            println("Make a move. Type 'resign' to resign from the match")
            userInput = readline()
            if(userInput == "resign")
                println("You resigned the game. The engine wins.")
                break
            end
            userMove = movefromstring(userInput)
            if(userMove == nothing)
                println("Invalid User Input. Please use the format [source] [destination]. Example(e2e4)")
                continue
            end
            if(!is_legal_move(game, userMove))
                println("Invalid Chess move. Please make a other move.")
                continue
            else
                current_boardscore = evaluate_move(board(game), userMove, current_boardscore)
                clearCache(board(game), userMove)
                domove!(game, userMove)
                # DEBUG 
                if debug && current_boardscore != evaluate_position(board(game))
                    println("Inkremental score is $(current_boardscore) but should be $(evaluate_position(board(game)))")
                end
            end
            printGame(game)
        else
            break
        end
        
        # Engine Move
        if !isterminal(game)
            # meassure time needed for the engine move
            @time begin
                current_boardscore = playMove(game, current_boardscore)
            end
            # DEBUG
            if debug && current_boardscore != evaluate_position(board(game))
                println("Inkremental score is $(current_boardscore) but should be $(evaluate_position(board(game)))")
            end
            printGame(game)
        else
            break
        end
    end
    setWin(game)
    saveGameToPGN(game)
    return game
end

playGame (generic function with 2 methods)

In [51]:
playGame(false)

Board (rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  P  P  P  P 
 R  N  B  Q  K  B  N  R

What's your name?
stdin> Karl
Do you want to play as 'white' or 'black'?
stdin> white
Make a move. Type 'resign' to resign from the match
stdin> e2e4
Cleared Cache with 0


Board (rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Evaluation of engine: 70.0
Current board score: 40
Current entries in Cache: 2941
Cleared Cache with 2941
  2.608660 seconds (8.53 M allocations: 464.209 MiB, 4.67% gc time)


Board (r1bqkbnr/pppppppp/2n5/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq -):
 r  -  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  n  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

...

Board (4kb1r/p1Q2bp1/2P1pp1p/1P6/P2P4/5N1P/5PPB/R3R1K1 w - -):
 -  -  -  -  k  b  -  r 
 p  -  Q  -  -  b  p  - 
 -  -  P  -  p  p  -  p 
 -  P  -  -  -  -  -  - 
 P  -  -  P  -  -  -  - 
 -  -  -  -  -  N  -  P 
 -  -  -  -  -  P  P  B 
 R  -  -  -  R  -  K  -

Make a move. Type 'resign' to resign from the match
stdin> c7d7
Cleared Cache with 0


Board (4kb1r/p2Q1bp1/2P1pp1p/1P6/P2P4/5N1P/5PPB/R3R1K1 b - -):
 -  -  -  -  k  b  -  r 
 p  -  -  Q  -  b  p  - 
 -  -  P  -  p  p  -  p 
 -  P  -  -  -  -  -  - 
 P  -  -  P  -  -  -  - 
 -  -  -  -  -  N  -  P 
 -  -  -  -  -  P  P  B 
 R  -  -  -  R  -  K  -

WHITE wins by checkmate[Event "AI Testing"]
[Site "DHBW Mannheim"]
[Date "2023.03.02"]
[Round "1"]
[White "Karl"]
[Black "Player 2"]
[Result "WHITE"]

1. e4 Nc6 2. d4 e6 3. Nf3 Nf6 4. e5 Ne4 5. Bd3 d5 6. exd6 Nxd6 7. O-O Nb4 8. Be2 Qf6 9. c3 Nd5 10. Bg5 Qf5 11. Bd3 Qg4 12. h3 Qh5 13. Nbd2 h6 14. Bh4 Nf4 15. Bc2 Nf5 16. Bxf5 Qxf5 17. Bg3 Ng6 18. Bxc7 Kd7 19. Bh2 Qb5 20. Qc2 Ne7 21. a4 Qb6 22. Nc4 Qa6 23. Nce5+ Ke8 24. b4 f6 25. b5 Qb6 26. Ng6 Nxg6 27. Qxg6+ Ke7 28. Rfe1 Bd7 29. c4 Be8 30. Qe4 Bf7 31. c5 Qa5 32. Qxb7+ Kd8 33. Qxa8+ Kd7 34. c6+ Ke7 35. Qb7+ Qc7 36. Qxc7+ Ke8 37. Qd7# WHITE



Game (Karl vs Player 2, DHBW Mannheim 2023):
 1. e4 Nc6 2. d4 e6 3. Nf3 Nf6 4. e5 Ne4 5. Bd3 d5 6. exd6 Nxd6 7. O-O Nb4 8. Be2 Qf6 9. c3 Nd5 10. Bg5 Qf5 11. Bd3 Qg4 12. h3 Qh5 13. Nbd2 h6 14. Bh4 Nf4 15. Bc2 Nf5 16. Bxf5 Qxf5 17. Bg3 Ng6 18. Bxc7 Kd7 19. Bh2 Qb5 20. Qc2 Ne7 21. a4 Qb6 22. Nc4 Qa6 23. Nce5+ Ke8 24. b4 f6 25. b5 Qb6 26. Ng6 Nxg6 27. Qxg6+ Ke7 28. Rfe1 Bd7 29. c4 Be8 30. Qe4 Bf7 31. c5 Qa5 32. Qxb7+ Kd8 33. Qxa8+ Kd7 34. c6+ Ke7 35. Qb7+ Qc7 36. Qxc7+ Ke8 37. Qd7# *